In [47]:
"""국토교통부_토지이용계획정보서비스
- 링크: https://www.data.go.kr/data/15056930/openapi.do?recommendDataYn=Y
"""
from dotenv import load_dotenv
import os
import math
import requests
import pandas as pd
import io
import xmltodict
import json
import tqdm

# import xml.etree.ElementTree as et


# load .env
load_dotenv()
getRTMSDataSvcAptTradeDev_en = os.environ.get("getRTMSDataSvcAptTradeDev_en")
getRTMSDataSvcAptTradeDev_de = os.environ.get("pubkey_de")

## Check total rows count

In [42]:
SIGUNGU_CODE = "11470"
B_CODE = "10200"

url = "http://apis.data.go.kr/1611000/nsdi/LandUseService/attr/getLandUseAttr"
params = {
    "serviceKey": getRTMSDataSvcAptTradeDev_de,
    "pnu": SIGUNGU_CODE + B_CODE,  # 필지 고유번호
    # "cnflcAt":  # 저촉여부코드(1:포함, 2:저촉, 3:접합)
    # "prposAreaDstrcCodeNm":  # 용도지역지구명
    "format": "xml",  # 응답결과 형식(xml 또는 json)
    "numOfRows": 10,  # 검색 건수
    "pageNo": 1,  # 페이지 번호
}

response = requests.get(url, params=params)
rows_count = xmltodict.parse(response.content)["response"]["totalCount"]
print(rows_count)
GET_ROWS_BY_ITER = 1000
dict_content = xmltodict.parse(response.content)["response"]
iter_range = math.ceil(int(rows_count) / GET_ROWS_BY_ITER)

54164


In [56]:
dict_list = []

for i in tqdm.tqdm(range(1, iter_range + 1)):
    params = {
        "serviceKey": getRTMSDataSvcAptTradeDev_de,
        "pnu": SIGUNGU_CODE + B_CODE,  # 필지 고유번호
        "format": "xml",  # 응답결과 형식(xml 또는 json)
        "numOfRows": GET_ROWS_BY_ITER,  # 검색 건수
        "pageNo": i,  # 페이지 번호
    }

    response = requests.get(url, params=params)
    try:
        dict_content = xmltodict.parse(response.content)["response"]["fields"]["field"]
        dict_list.extend(dict_content)
    except Exception as e:
        print(f"Error message {e}, Iter {i}")

100%|██████████| 55/55 [03:29<00:00,  3.81s/it]


In [59]:
df = pd.DataFrame(dict_list)
df.to_csv("data/서울특별시_양천구_토지이용계획_속성.csv",
          encoding="utf8",
          index=False
)